In [12]:
#!pip install nltk pandas nlpaug

In [13]:
import nlpaug.augmenter.word as naw
import pandas as pd
import os
from pandas import isnull
from collections import defaultdict
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cwwojin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
#Creating Profanity-spelling-error-dictionary (txt-file)
#2 files : profanity-only, combined
data_path = "datasets/"
spelling_path = "spelling/"

train_df = pd.read_csv(data_path + "2020-12-31-DynamicallyGeneratedHateDataset-entries-v0.1.csv", sep=',')[['text', 'label']]
profanity_df = pd.read_csv(data_path + "profanity_en.csv", sep=',')
labels = ['canonical_form_1', 'canonical_form_2', 'canonical_form_3']
spelling_dict = defaultdict(list)
fname_profanity = spelling_path + "spelling_en_profanity.txt"
#fname_combined = spelling_path + "spelling_en_combined.txt"

for idx, row in profanity_df.iterrows() :
    text = row['text']
    canons = [i for i in list(row[labels]) if not isnull(i)]
    for word in canons :
        spelling_dict[word].append(text)

if os.path.exists(fname_profanity):
    os.remove(fname_profanity)

with open(fname_profanity, "w+", encoding="UTF-8") as spelling_dict_file :
    for k, v in spelling_dict.items() :
        #print(" ".join([k] + v))
        spelling_dict_file.write(" ".join([k] + v))
        spelling_dict_file.write("\n")

# with open(fname_combined, "a+", encoding="UTF-8") as spelling_dict_file :
#     for k, v in spelling_dict.items() :
#         #print(" ".join([k] + v))
#         spelling_dict_file.write(" ".join([k] + v))
#         spelling_dict_file.write("\n")

In [15]:
#Modifying Original Dataset
profanities = list(spelling_dict.keys())
for idx, row in train_df.iterrows() :
    text = row['text']
    label = row['label']
    

In [4]:
def run_aug(n, N, train_df, aug, output_fname, path='') :
    train_df_sample = train_df.sample(n=N, ignore_index=True)
    #save sample as file
    train_df_sample.to_csv(path + output_fname[:-4] + "sample_{N}_.csv".format(N=N), sep=",")

    #aug.augment(data, n)
    augmented_data_sentences = []
    augmented_data_labels = []

    for idx, row in train_df_sample.iterrows() :
        sentence = row['text']
        label = row['label']
        augmented = aug.augment(sentence, n=n) if n > 1 else [aug.augment(sentence, n=n)]

        #label-preserving
        augmented_data_sentences += ([sentence] + augmented)
        augmented_data_labels += [label for i in range(n+1)]
    
    #save new dataset to csv-file
    output_df = pd.DataFrame(list(zip(augmented_data_sentences, augmented_data_labels)), columns=['sentence', 'label'])
    output_df.to_csv(path + output_fname, sep=",")

    return path + output_fname

In [5]:
#Data augmentation
stop_words = list(set(stopwords.words('english')))
aug_spelling_prof = naw.SpellingAug(dict_path=fname_profanity, stopwords=stop_words)

fnames = [("Dataset_aug_profanity_{i}_.csv").format(i=i) for i in [1000, 5000, 10000]]
output_path = "augmented_data/"

for fname in fnames :
    tokens = fname.split("_")
    N = int(tokens[3])
    print("Augmenting - N : {N}".format(N=N))
    run_aug(5, N, train_df, aug_spelling_prof, fname, path=output_path)

Augmenting - N : 1000
Augmenting - N : 5000
Augmenting - N : 10000
